In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
cd /content/drive/My Drive/NLP/fake-news

/content/drive/My Drive/NLP/fake-news


In [ ]:
import pandas as pd

Read Training data

In [ ]:
df=pd.read_csv('train.csv')

In [ ]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [ ]:
#Drop NA values
df=df.dropna()

In [ ]:
#Extract independent features
X=df.drop('label',axis=1)

In [ ]:
#Extract target variable
y=df['label']

In [ ]:
#Shape of independent features
X.shape

(18285, 4)

In [ ]:
#Shape of dependent features
y=df['label']

In [ ]:
X.shape

(18285, 4)

In [ ]:
y.shape

(18285,)

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [ ]:
#Vocabulary Size
voc_size=5000

In [ ]:
messages=X.copy()

In [ ]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [ ]:
messages.reset_index(inplace=True)

In [ ]:
import nltk
import re
from nltk.corpus import stopwords

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
#Data preprocessing - Generate corpus
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()
    
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [ ]:
print(len(corpus))

18285


In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus]

In [ ]:
print(onehot_repr[0])

[2615, 3459, 3150, 2481, 4487, 1791, 524, 3531, 1846, 2913]


Embedding Representation

In [ ]:
sent_length=20
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)

In [ ]:
print(embedded_docs[0])

[   0    0    0    0    0    0    0    0    0    0 2615 3459 3150 2481
 4487 1791  524 3531 1846 2913]


In [ ]:
#Creating the model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
len(embedded_docs),y.shape

(18285, (18285,))

In [ ]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(y)

In [ ]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.33, random_state=42)

In [ ]:
#Train the model
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

Epoch 1/10
192/192 [==============================] - 7s 36ms/step - loss: 0.3380 - accuracy: 0.8386 - val_loss: 0.2032 - val_accuracy: 0.9143
Epoch 2/10
192/192 [==============================] - 6s 32ms/step - loss: 0.1391 - accuracy: 0.9454 - val_loss: 0.1924 - val_accuracy: 0.9181
Epoch 3/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0910 - accuracy: 0.9669 - val_loss: 0.2331 - val_accuracy: 0.9022
Epoch 4/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0563 - accuracy: 0.9807 - val_loss: 0.2684 - val_accuracy: 0.9120
Epoch 5/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0316 - accuracy: 0.9896 - val_loss: 0.3287 - val_accuracy: 0.9118
Epoch 6/10
192/192 [==============================] - 6s 32ms/step - loss: 0.0227 - accuracy: 0.9931 - val_loss: 0.4388 - val_accuracy: 0.9067
Epoch 7/10
192/192 [==============================] - 6s 33ms/step - loss: 0.0145 - accuracy: 0.9951 - val_loss: 0.3694 - val_accuracy: 0.9021

Metrics calculation

In [ ]:
y_pred=model.predict_classes(X_test)

In [ ]:
print(len(y_pred))

6035


In [ ]:
from sklearn.metrics import confusion_matrix

In [ ]:
confusion_matrix(y_test,y_pred)

array([[3106,  313],
       [ 255, 2361]])

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test,y_pred)

0.9058823529411765